# Pandapipes Mapbox Plotting Tutorial
This notebook demonstrates how to use the Mapbox-based plotting functionality in pandapipes’s `simple_plot` function.

## 1. Setup
Ensure you have set the `MAPBOX_ACCESS_TOKEN` environment variable before running these cells. You can also override it in code if needed.

In [2]:
import os
# Verify that the token is available
token = os.getenv('MAPBOX_ACCESS_TOKEN')
assert token, 'Please set the MAPBOX_ACCESS_TOKEN environment variable'
from pandapipes.plotting.mapbox_plot import set_mapbox_token
set_mapbox_token(token)
print('Mapbox token is set.')

Mapbox token is set.


## 2. Create a Simple Network
We’ll build a two-consumer loop network for demonstration.

In [3]:
import pandapipes as pp
import pandas as pd

# Coordinates
coords_main = [
    (13.377704, 52.509669),  # j0: Plant
    (13.380000, 52.509700),  # j1
    (13.382000, 52.509720),  # j2
    (13.384000, 52.509740),  # j3
]
coords_h = {
    'h1_feed':   (13.380500, 52.510500),
    'h1_return': (13.380300, 52.510300),
    'h2_feed':   (13.382500, 52.510500),
    'h2_return': (13.382300, 52.510300),
}

# Build network
net = pp.create_empty_network(fluid='water', name='2-Consumer Loop')
main_j = [pp.create_junction(net, pn_bar=3, tfluid_k=353.15) for _ in coords_main]
j_feed1, j_ret1 = pp.create_junction(net, pn_bar=3, tfluid_k=353.15), pp.create_junction(net, pn_bar=3, tfluid_k=353.15)
j_feed2, j_ret2 = pp.create_junction(net, pn_bar=3, tfluid_k=353.15), pp.create_junction(net, pn_bar=3, tfluid_k=353.15)

# Assign geodata
all_coords = coords_main + list(coords_h.values())
all_juncs  = main_j + [j_feed1, j_ret1, j_feed2, j_ret2]
net.junction_geodata = pd.DataFrame({
    'lon': [lon for lon, lat in all_coords],
    'lat': [lat for lon, lat in all_coords],
}, index=all_juncs)

# Pipes and devices
for i in range(len(main_j)-1):
    pp.create_pipe_from_parameters(net, main_j[i], main_j[i+1], length_km=0.1, diameter_m=0.1,
                                   k_mm=0.02, alpha_w_per_m2k=10, text_k=293.15, name=f'loop_{i}')
pp.create_pipe_from_parameters(net, main_j[1], j_feed1,   0.02, 0.05, k_mm=0.02, alpha_w_per_m2k=10, text_k=293.15, name='h1_supp')
pp.create_pipe_from_parameters(net, j_ret1,   main_j[2], 0.02, 0.05, k_mm=0.02, alpha_w_per_m2k=10, text_k=293.15, name='h1_ret')
pp.create_pipe_from_parameters(net, main_j[2], j_feed2,   0.02, 0.05, k_mm=0.02, alpha_w_per_m2k=10, text_k=293.15, name='h2_supp')
pp.create_pipe_from_parameters(net, j_ret2,   main_j[3], 0.02, 0.05, k_mm=0.02, alpha_w_per_m2k=10, text_k=293.15, name='h2_ret')
pp.create_heat_exchanger(net, from_junction=j_feed1, to_junction=j_ret1, qext_w=15e3, diameter_m=0.05,
                         length_km=0.005, sections=2, name='HX1')
pp.create_heat_exchanger(net, from_junction=j_feed2, to_junction=j_ret2, qext_w=25e3, diameter_m=0.05,
                         length_km=0.005, sections=2, name='HX2')
pp.create_ext_grid(net, junction=main_j[0], p_bar=3.0, t_k=353.15, name='Plant')
pp.create_circ_pump_const_mass_flow(net, return_junction=main_j[-1], flow_junction=main_j[0],
                                     mdot_flow_kg_per_s=0.1, p_flow_bar=3.5, t_flow_k=353.15, name='LoopPump')
pp.pipeflow(net, mode='sequential', friction_model='swamee-jain')

diameter_m is deprecated as it has no effect on the calculation and results. Nonetheless, it will be stored in the compoent table for postprocessing purposes by you if required.
diameter_m is deprecated as it has no effect on the calculation and results. Nonetheless, it will be stored in the compoent table for postprocessing purposes by you if required.


## 3. Plotting with Mapbox
Use `simple_plot` with `use_mapbox=True` and customize parameters:

In [9]:
from pandapipes.plotting.simple_plot import simple_plot

fig = simple_plot(
    net,
    use_mapbox=True,
    map_style='light',  # e.g. 'streets', 'dark', 'satellite'
    zoom=15,
    pipe_temperature_coloring=True,
    show_colorbar=True,
    pipe_temperature_colorscale=[
        [0.0, 'blue'],
        [0.5, 'yellow'],
        [1.0, 'red'],
    ],
    renderer='browser'
)
fig.show()

<img src="pics/mapbox_plot_sample.png">

### Parameters Reference
- **use_mapbox** (bool): Switch to Mapbox backend (default: False)
- **mapbox_access_token** (str): Your Mapbox token (env var or override)
- **map_style** (str): Mapbox style, e.g., 'streets', 'light', 'dark'
- **zoom** (int): Initial zoom level
- **pipe_temperature_coloring** (bool): Color pipes by temperature
- **pipe_temperature_field** (str): Column in `net.res_pipe` (default: 't_to_k')
- **pipe_temperature_colorscale** (list): Color stops for temperature scale
- **show_colorbar** (bool): Display the colorbar
- **renderer** (str): Plotly renderer, e.g., 'browser', 'notebook'


